# Import dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import date

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
from custom_modules import descriptive_analysis as da
from custom_modules import analysis as a
from custom_modules import feature_engineering as fe
from custom_modules import plotting as pl

# Input Files

In [4]:
file_path_1 = r"gs://ppl-analytics-cfs/Survival Ad-hoc/2022 Adjustment eff February 20, 2022 revised.xlsx"
file_path_2 = r"gs://ppl-analytics-cfs/Survival Ad-hoc/Copy of DemographicData.csv"
file_path_3 = r"gs://ppl-analytics-cfs/Survival Ad-hoc/adjustment -June 2022.xlsx"
file_path_4 = r"gs://ppl-analytics-cfs/Survival Ad-hoc/Book2.xlsx"
file_path_5 = r"gs://ppl-analytics-cfs/Survival Ad-hoc/adj_june.xlsx"

In [ ]:
df_past = pd.read_excel(file_path_1)
df_colleague = pd.read_csv(file_path_2, low_memory=False)
df_current = pd.read_excel(file_path_5)
df_pc = pd.read_excel(file_path_4)
# new_june = pd.read_excel(file_path_5)

# EDA

## Colleague Data EDA

In [ ]:
proper_data_types = {'Active Status':'category',
                     'Base Pay Eff Date':'datetime64',
                     'Benefits Service Date':'datetime64',
                     'Bonus Plan Profile Target Percent':'float32',
                     'Compa Ratio':'float32',
                     'Compensation Grade':'category',
                     'Compensation Most Recent Change Date':'datetime64',
                     'Compensation Package':'category',
                     'Compensation Plan Type':'category',
                     'Country':'category',
                     'Default Weekly Hours':'float32',
                     'Date of Birth':'datetime64',
                     'Dist Channel Group':'category',
                     'Effective Date for Current Position':'datetime64',
                     'Employee ID':'int32',
                     'Employee Type':'category',
                     'Estimated Last Day of Leave':'datetime64',
                     'FTE':'float64',
                     'First Day of Leave':'datetime64',
                     'Full Time Date/Company Serv Date':'datetime64',
                     'Gender':'category',
                     'Hire Date':'datetime64',
                     'Hire Corp Year Num':'float32',
                     'Hire Corp Week Num':'float32',
                     'Hours Per Shift':'category',
                     'Is Manager':'category',
                     'Job Family Group':'category',
                     'Job Profile Start Date':'datetime64',
                     'Language':'category',
                     'Last Day of Work for Leave':'category',
                     'Location - Scheduled Hours':'category',
                     'Location Hierarchy':'category',
                     'Management Level':'category',
                     'Meal Flag':'category',
                     'On Leave':'category',
                     'Original Hire Date':'datetime64',
                     'Part-Time Availability':'category',
                     'Pay Frequency - Payroll':'category',
                     'Pay Group':'category',
                     'Pay Rate Type - Job Profile':'category',
                     'Pay Rate Type - Worker':'category',
                     'Pay Status':'category',
                     'Pay Through Date':'datetime64',
                     'Province':'category',
                     'Region':'category',
                     'Salary Continuance':'datetime64',
                     'Scheduled Weekly Hours':'float32',
                     'Scheduling Date':'datetime64',
                     'Sen Date':'datetime64',
                     'Seniority Date':'datetime64',
                     'Staffing Model':'category',
                     'Store Number':'category',
                     'Structure':'category',
                     'INTCF - 032 - Supervisory Org Subtype':'category',
                     'Term less than 12 months':'category',
                     'Term less than 6 months':'category',
                     'Terminated':'category',
                     'Termination Corp Year Num':'float32',
                     'Termination Corp Week Num':'float32',
                     'Termination Category':'category',
                     'Termination Date':'datetime64',
                     'Termination Reason':'category',
                     'Time Off Service Date':'datetime64',
                     'Time Profile':'category',
                     'Time Type':'category',
                     'Time in Position':'float32',
                     'Total Base Pay - Amount':'float32',
                     'Total Base Pay - Frequency':'category',
                     'Union':'category',
                     'Union?':'category',
                     'Vacation Date/Cont Serv Date':'datetime64',
                     'Yrs of Service as of Dec 31st':'float32',
                     'zWD_SITE':'category'
                    }

for key in set(proper_data_types.keys()).difference(df_colleague.columns): 
       del proper_data_types[key]
proper_data_types = da.missing_dict(proper_data_types)
df_colleague = df_colleague.astype(proper_data_types, errors='ignore')
        
text_cols = [col for col in df_colleague.columns if df_colleague[col].dtype == 'object']

df_colleague = fe.nlp.make_lower(df_colleague, text_cols)
df_colleague = fe.nlp.remove_extra_spaces(df_colleague, text_cols)
df_colleague = df_colleague.rename(columns = {'INTCF - 032 - Supervisory Org Subtype':'Supervisory Org Subtype'})

In [ ]:
da.display_description(df_colleague)

In [ ]:
da.duplicated_rows(df_colleague, 'ID - Win', output=False)

In [ ]:
df_colleague = df_colleague.drop_duplicates(subset = ['ID - Win'])

In [ ]:
df_colleague.head()

## Adjustment Data

In [ ]:
past_data_types = {'2021 Top Talent':'category',
                     'Adjustment_Diff':'float32',
                     'Business Unit':'category',
                     'Categorization Flag':'category',
                     'Comp Validated Prioritzation Segment':'category',
                     'Competency in Role (based on current role)':'category',
                     'Current Annualized Rate':'float32',
                     'Current Internal Ratio':'float32',
                     'Current Market Ratio':'float32',
                     'Current Salary':'float32',
                     'Employee ID':'int32',
                     'Final EIB Adjustment':'float32',
                     'Final EIB Annualized Salary':'float32',
                     'Final Internal Ratio':'float32',
                     'Final Market Ratio':'float32',
                     'GPE Priority Categorization - Org Wide':'category',
                     'Gender':'category',
                     'Grade':'category',
                     'HIPO + HIPER':'category',
                     'Hot Skill':'category',
                     'Internal Reference':'float32',
                     'Job Profile':'object',
                     'Management Board':'object',
                     'Market Reference':'float32',
                     'Market Relativity':'category',
                     'Market reference annualized':'float32',
                     'PDT FLAG CATEGORIZATION':'category',
                     'Performance':'category',
                     'Potential':'category',
                     'Received Adjustment?':'category',
                     'Recommended Adjustment':'float32',
                     'Recommended Salary/Wage':'float32',
                     'Retention':'category',
                     'Supervisory Org':'category',
                     'Supervisory Org Channel':'category',
                     'Time in Role (Years)':'float32'}

proper_data_types = da.missing_dict(past_data_types)
df_past = df_past.astype(past_data_types, errors='ignore')

In [ ]:
proper_data_types_current = da.missing_dict(proper_data_types)
df_current = df_current.astype(proper_data_types_current, errors = 'ignore')

In [ ]:
df_current['Cost - July 29 Calc'].value_counts()

In [ ]:
df_current['Cost Centre Hierarchy Top Lev 06']

In [ ]:
da.display_description(df_past)

In [ ]:
df_past = df_past.dropna(subset = ['Employee ID'])

In [ ]:
da.duplicated_rows(df_past, 'Employee ID', output=False)

In [ ]:
df_past = df_past.drop_duplicates(subset = ['Employee ID'])

## Current Adjustment Data

In [ ]:
df_current

# Merge Data Frames

In [ ]:
df_merged = df.merge(df_colleague[['ID - Win', 'Hire Date','Terminated', 'Termination Date','Termination Category']], left_on = 'Employee ID' , right_on = 'ID - Win', how = 'left')

In [ ]:
filter_1 = (df_merged['Termination Category'] != 'Involuntary')

In [ ]:
df_final = df_merged[(filter_1)]

In [ ]:
print(df.shape)
print(df_colleague.shape)
print(df_merged.shape)
print(df_final.shape)

# Feature Engineering

In [ ]:
# # Converting terminated to binary column for WOE calculations

df_colleague['Terminated'] = df_colleague['Terminated'].astype(str).map({'Yes':1}).fillna(0).astype('int32')

## Colleague Data

### Tenure

In [ ]:
# Creating a reference data for calculating age and tenurity (if terminated, termination date otherwise today's date)
df_colleague['Reference Date'] = df_colleague['Termination Date']
df_colleague['Reference Date'] = df_colleague['Reference Date'].fillna(str(date.today()))
df_colleague['Reference Date'] = pd.to_datetime(df_colleague['Reference Date']) 

df_colleague['Tenure'] = round(fe.drivers.num_month_diff(df_colleague['Reference Date'].dt, df_colleague['Hire Date'].dt) / 12)
df_colleague['Tenure'] = df_colleague['Tenure'].astype('float32')

# Calculated tenure based on original hire date is considred for analysis instead of time in position
print("Percentage of colleagues with the same time in position and tenure: {:.2f}%".format(sum(df_colleague['Time in Position'] == df_colleague['Tenure'])/len(df_colleague) * 100))

# Calculated tenure based on original hire date is considred for analysis instead of Yrs of Service as of Dec 31st
print("Percentage of colleagues with the same years of Service as of Dec 31st and tenure: {:.2f}%" \
      .format(sum(df_colleague['Yrs of Service as of Dec 31st'] == df_colleague['Tenure'])/len(df_colleague) * 100))

### Age

In [ ]:
df_colleague['Age'] = round(fe.drivers.num_month_diff(df_colleague['Reference Date'].dt, df_colleague['Date of Birth'].dt) / 12)
df_colleague['Age'] = df_colleague['Age'].astype('float32')

In [ ]:
# Required functions
def optimal_bins(df, ind_col, target_col, plot_title, data_type='numerical', binning_solver='cp', table_out=True, plot_out=True, analysis_out=True):
    """Finding optimal bins for categorization of results and generating a table, a plot, and analysis report"""
    from optbinning import OptimalBinning
    x = df[ind_col].values
    y = df[target_col]
    optb = OptimalBinning(name=plot_title, dtype=data_type, solver=binning_solver)
    optb.fit(x, y)
    binning_table = optb.binning_table
    if table_out: display(binning_table.build())
    if plot_out: binning_table.plot()
    if analysis_out: display(binning_table.analysis())
    if table_out: return binning_table.build()

def bin_labels(bins_list):
    """
    Creating a list of labels based on bin values for plotting
    Note: The space before '<' is on purpose to be able to sort values in Excel properly. 
    """
    labels_list = []
    labels_list.append(' <' + str(bins_list[1]))
    for i in range(1, len(bins_list)-2):
        labels_list.append(str(bins_list[i]) + ' - ' + str(bins_list[i+1]))
    labels_list.append('>' + str(bins_list[-2]))
    return labels_list

def cat_plot(df, list_cols=None, target_col=None, plot_kind='strip', horizontal=True, pos_neg_color=True, 
             title='Categorical Plot for ', title_size=10, estimator=np.mean, confd_int=None):
    """
    Ploting the distribution of categorical cols based on values of a target column.
    Availalbe plot kinds: 'strip', 'swarm', 'box', 'violin', 'boxen', 'point', 'bar', or 'count'
    """
    from seaborn import catplot, set, set_style
    import numpy as np
    # Checking columns
    if list_cols is None: list_cols = [col for col in df.columns if str(df[col].dtype) == 'category']
    if target_col is None: target_col = df.iloc[:,0].name
    # Setting font size
    set(font_scale = 0.05 * title_size + 0.35)
    set_style("ticks")
    # Setting colors
    if pos_neg_color:
        colors = np.where(df[target_col] < 0, 'red', 'green')
    else:
        colors = 'bright'
    # Creating melted dataframe for multiple rows plotting
    df = df[list_cols + [target_col]].melt(id_vars=target_col, value_vars=list_cols, var_name='Attribute', value_name='Category', ignore_index=True)
    # Plotting
    if horizontal:
        g = catplot(x=target_col, y='Category', col='Attribute', data=df, kind=plot_kind, orient="h", palette=colors, color=colors, col_wrap=4, 
                    ci=confd_int, n_boot=1000, seed=42, legend=False, legend_out=True, estimator=estimator, sharey=False, sharex=False)
        g.set_titles(title + "{col_name}", size=title_size)
    else:
        g = catplot(x='Category', y=target_col, col='Attribute', data=df, kind=plot_kind, orient="v", palette=colors, color=colors, col_wrap=4, 
                    ci=confd_int, n_boot=1000, seed=42, legend=False, legend_out=True, estimator=estimator, sharey=False, sharex=False)
        g.set_titles(title + "{col_name}", size=title_size)

### Age

In [ ]:
col_name = 'Age'
pl.histogram_plot(df_colleague, list_cols=[col_name], agg_stat='count', label_size=12, figure_size=(12, 5), bins_method='auto', kernel_density=False)

In [ ]:
binning_table = optimal_bins(df_colleague, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                 data_type='numerical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Creating categorized column
bins_list = [0, 25, 35, 50, 58, 100]
labels_list = bin_labels(bins_list)

df_colleague[col_name + ' Category'] = pd.cut(df_colleague[col_name], bins=bins_list, labels=labels_list, right=False, retbins=False, precision=3, ordered=True)

df_woe = a.reg_analysis.getWOE(df_colleague, col_name + ' Category', 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name + ' Category'], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Tenure

In [ ]:
col_name = 'Tenure'

In [ ]:
pl.histogram_plot(df, list_cols=[col_name], agg_stat='count', label_size=12, figure_size=(12, 5), bins_method='auto', kernel_density=False)

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='numerical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Creating categorized column
bins_list = [0, 0.5, 2.5, 6.5, 10.5, 65]
labels_list = bin_labels(bins_list)

df[col_name + ' Category'] = pd.cut(df[col_name], bins=bins_list, labels=labels_list, right=False, retbins=False, precision=3, ordered=True)

df_woe = anl.reg_analysis.getWOE(df, col_name + ' Category', 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name + ' Category'], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Base Pay (Salary)

In [ ]:
col_name = 'Total Base Pay - Amount'

In [ ]:
pl.histogram_plot(df, list_cols=[col_name], agg_stat='count', label_size=12, figure_size=(12, 5), bins_method='auto', kernel_density=False)

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='numerical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Creating categorized column
bins_list = [0, 16, 18.2, 22.4, 24.8, 26, 40]
labels_list = bin_labels(bins_list)

df[col_name + ' Category'] = pd.cut(df[col_name], bins=bins_list, labels=labels_list, right=False, retbins=False, precision=3, ordered=True)
df_req = df_req.rename(columns={'Total Base Pay - Amount Category':'Base Pay Category'})

df_woe = anl.reg_analysis.getWOE(df, 'Base Pay Category', 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=['Base Pay Category'], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Job Functions

In [ ]:
col_name = 'Job Functions'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Scheduled Weekly Hours

In [ ]:
col_name = 'Scheduled Weekly Hours'

In [ ]:
pl.histogram_plot(df, list_cols=[col_name], agg_stat='count', label_size=12, figure_size=(12, 5), bins_method='auto', kernel_density=False)

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='numerical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Creating categorized column
bins_list = [0, 22, 38, 50]
labels_list = bin_labels(bins_list)

df[col_name + ' Category'] = pd.cut(df[col_name], bins=bins_list, labels=labels_list, right=False, retbins=False, precision=3, ordered=True)

df_woe = anl.reg_analysis.getWOE(df, col_name + ' Category', 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name + ' Category'], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Time Type

In [ ]:
col_name = 'Time Type'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Employee Type

In [ ]:
col_name = 'Employee Type'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Gender

In [ ]:
col_name = 'Gender'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Pay Group

In [ ]:
col_name = 'Pay Group'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df.loc[df[col_name].isin(['Loblaw_Bi-weekly','Loblaw_Weekly'])].copy(), col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Promotion

In [ ]:
col_name = 'Promotion'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Other Attributes

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("Compensation Plan")
sorted(df['Compensation Plan'].astype(str).unique())

In [ ]:
# Benefits Service Date
col_name = 'Benefits Service Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Original Hire Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Original Hire Date']).describe())

In [ ]:
# Compensation Most Recent Change Date
col_name = 'Compensation Most Recent Change Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Termination Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Termination Date']).describe())

In [ ]:
# First Day of Leave
col_name = 'First Day of Leave'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Termination Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Termination Date']).describe())

In [ ]:
# Full Time Date/Company Serv Date
col_name = 'Full Time Date/Company Serv Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Original Hire Date:")
print((df.loc[df[col_name].notna(), col_name] != df.loc[df[col_name].notna(), 'Original Hire Date']).describe())

In [ ]:
# Job Profile Start Date
col_name = 'Job Profile Start Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Original Hire Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Original Hire Date']).describe())

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("Meal Flag")
sorted(df['Meal Flag'].astype(str).unique())

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("On Leave")
sorted(df['On Leave'].astype(str).unique())

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("Supervisory Org Subtype")
sorted(df['Supervisory Org Subtype'].astype(str).unique())

In [ ]:
# Indicator of promotion based on Effective Date for Current Position
df['Promotion'] = (df['Effective Date for Current Position'] > df['Original Hire Date']).astype('category')

In [ ]:
# Required functions
def optimal_bins(df, ind_col, target_col, plot_title, data_type='numerical', binning_solver='cp', table_out=True, plot_out=True, analysis_out=True):
    """Finding optimal bins for categorization of results and generating a table, a plot, and analysis report"""
    from optbinning import OptimalBinning
    x = df[ind_col].values
    y = df[target_col]
    optb = OptimalBinning(name=plot_title, dtype=data_type, solver=binning_solver)
    optb.fit(x, y)
    binning_table = optb.binning_table
    if table_out: display(binning_table.build())
    if plot_out: binning_table.plot()
    if analysis_out: display(binning_table.analysis())
    if table_out: return binning_table.build()

def bin_labels(bins_list):
    """
    Creating a list of labels based on bin values for plotting
    Note: The space before '<' is on purpose to be able to sort values in Excel properly. 
    """
    labels_list = []
    labels_list.append(' <' + str(bins_list[1]))
    for i in range(1, len(bins_list)-2):
        labels_list.append(str(bins_list[i]) + ' - ' + str(bins_list[i+1]))
    labels_list.append('>' + str(bins_list[-2]))
    return labels_list

def cat_plot(df, list_cols=None, target_col=None, plot_kind='strip', horizontal=True, pos_neg_color=True, 
             title='Categorical Plot for ', title_size=10, estimator=np.mean, confd_int=None):
    """
    Ploting the distribution of categorical cols based on values of a target column.
    Availalbe plot kinds: 'strip', 'swarm', 'box', 'violin', 'boxen', 'point', 'bar', or 'count'
    """
    from seaborn import catplot, set, set_style
    import numpy as np
    # Checking columns
    if list_cols is None: list_cols = [col for col in df.columns if str(df[col].dtype) == 'category']
    if target_col is None: target_col = df.iloc[:,0].name
    # Setting font size
    set(font_scale = 0.05 * title_size + 0.35)
    set_style("ticks")
    # Setting colors
    if pos_neg_color:
        colors = np.where(df[target_col] < 0, 'red', 'green')
    else:
        colors = 'bright'
    # Creating melted dataframe for multiple rows plotting
    df = df[list_cols + [target_col]].melt(id_vars=target_col, value_vars=list_cols, var_name='Attribute', value_name='Category', ignore_index=True)
    # Plotting
    if horizontal:
        g = catplot(x=target_col, y='Category', col='Attribute', data=df, kind=plot_kind, orient="h", palette=colors, color=colors, col_wrap=4, 
                    ci=confd_int, n_boot=1000, seed=42, legend=False, legend_out=True, estimator=estimator, sharey=False, sharex=False)
        g.set_titles(title + "{col_name}", size=title_size)
    else:
        g = catplot(x='Category', y=target_col, col='Attribute', data=df, kind=plot_kind, orient="v", palette=colors, color=colors, col_wrap=4, 
                    ci=confd_int, n_boot=1000, seed=42, legend=False, legend_out=True, estimator=estimator, sharey=False, sharex=False)
        g.set_titles(title + "{col_name}", size=title_size)

### Scheduled Weekly Hours

In [ ]:
col_name = 'Scheduled Weekly Hours'

In [ ]:
pl.histogram_plot(df, list_cols=[col_name], agg_stat='count', label_size=12, figure_size=(12, 5), bins_method='auto', kernel_density=False)

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='numerical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Creating categorized column
bins_list = [0, 22, 38, 50]
labels_list = bin_labels(bins_list)

df[col_name + ' Category'] = pd.cut(df[col_name], bins=bins_list, labels=labels_list, right=False, retbins=False, precision=3, ordered=True)

df_woe = anl.reg_analysis.getWOE(df, col_name + ' Category', 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name + ' Category'], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Time Type

In [ ]:
col_name = 'Time Type'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Employee Type

In [ ]:
col_name = 'Employee Type'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Gender

In [ ]:
col_name = 'Gender'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Pay Group

In [ ]:
col_name = 'Pay Group'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df.loc[df[col_name].isin(['Loblaw_Bi-weekly','Loblaw_Weekly'])].copy(), col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Promotion

In [ ]:
col_name = 'Promotion'

In [ ]:
binning_table = optimal_bins(df, ind_col=col_name, target_col='Terminated', plot_title=col_name + ' as Driver of Attrition', 
                                    data_type='categorical', binning_solver='cp', table_out=True, plot_out=False, analysis_out=False)

In [ ]:
# Calculating WoE
df_woe = anl.reg_analysis.getWOE(df, col_name, 'Terminated').reset_index().rename(columns={'woe':'WoE'})
display(df_woe)

cat_plot(df_woe, list_cols=[col_name], target_col='WoE', plot_kind='bar', horizontal=True, pos_neg_color=True, title='Attrition based on ', title_size=12)

### Other Attributes

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("Compensation Plan")
sorted(df['Compensation Plan'].astype(str).unique())

In [ ]:
# Benefits Service Date
col_name = 'Benefits Service Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Original Hire Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Original Hire Date']).describe())

In [ ]:
# Compensation Most Recent Change Date
col_name = 'Compensation Most Recent Change Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Termination Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Termination Date']).describe())

In [ ]:
# First Day of Leave
col_name = 'First Day of Leave'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Termination Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Termination Date']).describe())

In [ ]:
# Full Time Date/Company Serv Date
col_name = 'Full Time Date/Company Serv Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Original Hire Date:")
print((df.loc[df[col_name].notna(), col_name] != df.loc[df[col_name].notna(), 'Original Hire Date']).describe())

In [ ]:
# Job Profile Start Date
col_name = 'Job Profile Start Date'
print(col_name)
print("Missing: {} ({:.2f}%)".format(sum(df[col_name].isna()), sum(df[col_name].isna()) / len(df) * 100))
print("Difference between", col_name, "and Original Hire Date:")
print((df.loc[df[col_name].notna(), col_name] - df.loc[df[col_name].notna(), 'Original Hire Date']).describe())

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("Meal Flag")
sorted(df['Meal Flag'].astype(str).unique())

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("On Leave")
sorted(df['On Leave'].astype(str).unique())

In [ ]:
# Compensation Plan (all hourly based on the scope of colleagues)
print("Supervisory Org Subtype")
sorted(df['Supervisory Org Subtype'].astype(str).unique())

#### Past Adjustment Data

In [ ]:
da.display_description(df_past)

In [ ]:
df['Adjustment Difference'] = df['Adjustment_Diff']*100

In [ ]:
df = df[~df['Employee ID'].isna()]

In [ ]:
da.display_description(df)

In [ ]:
df[df['Grade'].str.contains('Hourly') == True][['Market Reference','Current Salary','Final EIB Adjustment',
       'Final EIB Annualized Salary', 'Final Market Ratio',
       'Final Internal Ratio', 'Adjustment_Diff', 'Recommended Salary/Wage',
       'Recommended Adjustment']]

In [ ]:
df['Market Reference Annualized 2'] = np.where(df['Grade'].str.contains('Hourly'), df['Market Reference']*280, df['Market Reference'])

In [ ]:
df[['Market reference annualized', 'Market Reference Annualized 2']]

In [ ]:
df[(df['Grade'].str.contains('Hourly')) & (~df['Grade'].isna())][['Current Salary', 'Current Annualized Rate','Final EIB Annualized Salary']]

In [ ]:
df_colleague.columns

# Feature Engineering

In [ ]:
df_final['Terminated'] = np.where(df_final['Terminated'] == 'Yes', 1, 0)
df_final['2021 Top Talent'] = np.where(df_final['2021 Top Talent'] == 'Yes', 1, 0)
df_final['HIPO + HIPER'] = np.where(df_final['HIPO + HIPER'] == 'Yes', 1, 0)
df_final['Hot Skill'] = np.where(df_final['Hot Skill'] == 'Yes', 1, 0)

In [ ]:
df_final['Final Market Ratio'] = df_final['Final EIB Annualized Salary']/df_final['Market reference annualized']*100

In [ ]:
df_final['Final Market Ratio']

In [ ]:
df_grouped_mean = df_final.groupby('Grade').agg({'Current Annualized Rate':'mean','Terminated':'sum','Adjustment Difference':'mean','Final Market Ratio':'mean'}).reset_index()

In [ ]:
df_grouped_mean

In [ ]:
# df_grouped_mean = df_grouped_mean[~df_grouped_mean['Final Market Ratio'].isna()]
df_grouped_mean

In [ ]:
df_final['Adjustment Difference'].max()

# WoE

In [ ]:
df_final['Adjustment Difference']

In [ ]:
df_final['Adjustment Difference'].hist()

In [ ]:
adj_bin = [-100,0.2,3,1000]
adj_bin_names = ['<0.2','0.2-3','>3']
df_final['adj_cat'] = pd.cut(df_final['Adjustment Difference'], bins = adj_bin, labels = adj_bin_names)

In [ ]:
df_woe_adjust = a.reg_analysis.getWOE(df_final, 'adj_cat', 'Terminated')
df_woe_adjust

In [ ]:
df_woe_adjust = a.reg_analysis.getWOE(df_final, 'adj_cat', 'Terminated')
df_woe_adjust

In [ ]:
da.list_unique_values(df_final,['Potential'])

In [ ]:
df_mratio = df_final[df_final['Final Market Ratio'] != np.inf]

In [ ]:
df_mratio['Final Market Ratio'].hist()

In [ ]:
market_bin = [-100,80,95,130,1000]
market_bin_names = ['<80','80-100','100-130','>130']
df_mratio['market_cat'] = pd.cut(df_mratio['Final Market Ratio'], bins = market_bin, labels = market_bin_names)
df_final['market_cat'] = pd.cut(df_final['Final Market Ratio'], bins = market_bin, labels = market_bin_names)

In [ ]:
df_woe_market = a.reg_analysis.getWOE(df_final, 'market_cat', 'Terminated')

# Aggregation

In [ ]:
len(df_final[(df_final['Received Adjustment?'] == 1) & ((df_final['Terminated'] == 1))])

In [ ]:
df_final[(df_final['Received Adjustment?'] == 1) & ((df_final['Terminated'] == 1))].groupby('Grade').agg({'Adjustment Difference':'mean','Employee ID':'count'})

In [ ]:
len(df_final[(df_final['Received Adjustment?'] == 0) & ((df_final['Terminated'] == 1))])

In [ ]:
df_final[(df_final['Received Adjustment?'] == 1) & ((df_final['Terminated'] == 0))].groupby('Grade').agg({'Adjustment Difference':'mean','Employee ID':'count'})

In [ ]:
len(df_final[(df_final['Received Adjustment?'] == 0) & ((df_final['Terminated'] == 0))])

In [ ]:
da.list_unique_values(df_final, ['Potential','Performance'])

In [ ]:
df_final.groupby('Potential').agg({'Terminated':'count'})

In [ ]:
df_perform = df_final.groupby('Performance').agg({'Employee ID':'count','Terminated':'sum','HIPO + HIPER':'mean','Hot Skill':'mean','2021 Top Talent':'mean','Adjustment Difference':'mean'})
df_perform['Termination %'] = df_perform['Terminated']/df_perform['Employee ID']*100
display(df_perform)

In [ ]:
df_potential = df_final.groupby('Potential').agg({'Employee ID':'count','Terminated':'sum','HIPO + HIPER':'mean','Hot Skill':'mean','2021 Top Talent':'mean','Adjustment Difference':'mean'})
df_potential['Termination %'] = df_potential['Terminated']/df_potential['Employee ID']*100
display(df_potential)

In [ ]:
table = pd.pivot_table(df_final, values = 'Terminated', index = ['Potential'], columns =['Performance'], aggfunc = np.mean)
table

# Logistic Regression

In [ ]:
da.list_unique_values(df_final,['Potential'])

In [ ]:
df_final['Terminated'].value_counts()

In [ ]:
df_final['Potential Cat'] = df_final['Potential'].map({'Too Early to Rank':'Too Early to Rank','Low':'Low Potential','Medium':'Medium Potential','High':'High Potential'})
# df_final['performance_cat'] = df_final['Performance'].map({'Developing Contributor':'D','Exceptional Contributor':'E','Not Performing':'N','Successful Contributor':'S'})

In [ ]:
df_merged_LReg = pd.DataFrame()

market_bin = [-100,80,95,130,1000]
market_bin_names = ['<80','80-100','100-130','>130']

adj_bin = [-100,0.2,3,1000]
adj_bin_names = ['<0.2','0.2-3','>3']

df_final['Market Ratio Cat'] = pd.cut(df_final['Final Market Ratio'], bins = market_bin, labels = market_bin_names)
df_final['Adjustment Ratio Cat'] = pd.cut(df_final['Adjustment Difference'], bins = adj_bin, labels = adj_bin_names)

featureForLogReg = ['Market Ratio Cat','Adjustment Ratio Cat','Potential Cat','Performance']
df_forClustering = df_final[featureForLogReg+['Terminated']+['Employee ID']]

In [ ]:
df_forClustering['Terminated'].value_counts()

In [ ]:
df_forClustering

In [ ]:
for col in featureForLogReg:
        df_forClustering = df_forClustering.join(pd.get_dummies(df_forClustering[col])) 
df_forClustering = df_forClustering.drop(featureForLogReg, axis=1)

In [ ]:
df_forClustering_train,df_forClustering_test = train_test_split(df_forClustering,train_size = 0.3, random_state = 100)
df_forClustering_trainset,df_forClustering_validate = train_test_split(df_forClustering_train,train_size = 0.8, random_state = 101)

df_forClustering_x = df_forClustering_trainset.drop(['Employee ID','Terminated'], axis = 1)
df_forClustering_y = df_forClustering_trainset[['Terminated']]
    
clf = LogisticRegression(random_state=0).fit(df_forClustering_x, df_forClustering_y)

In [ ]:
clf.score(df_forClustering_validate.drop(['Employee ID','Terminated'], axis=1),
          df_forClustering_validate['Terminated'])

In [ ]:
df_forClustering.columns

In [ ]:
prob = clf.predict_proba(df_forClustering.drop(['Employee ID','Terminated'], axis=1))
prob = [x for (x,y) in prob]

df_forClustering['Termination Probability'] = prob
df_forClustering
# listOfActiveColleagues[['Employee ID','teminationProbability']].to_csv('employee_level_teminationProbability.csv')

In [ ]:
df_forClustering_order = df_forClustering.sort_values(by =['Termination Probability'], ascending = False).reset_index()

In [ ]:
df_forClustering_order['Rank'] = pd.qcut(df_forClustering_order.index, 5, labels = False)

In [ ]:
df_forClustering_order.groupby('Rank').agg({'Termination Probability':'min','Employee ID':'count'})

In [ ]:
df_forClustering_order.groupby('Rank').agg({'Termination Probability':'max','Employee ID':'count'})

In [ ]:
df_probability = df_forClustering_order.groupby(by =['Rank']).agg({'Employee ID':'count'})

In [ ]:
df_forClustering_order_rev = df_forClustering.sort_values('Termination Probability', ascending = True).reset_index()
df_forClustering_order_rev

In [ ]:
df_forClustering_order_rev['Rank'] = pd.qcut(df_forClustering_order_rev.index, 5, labels = False)
df_forClustering_order_rev.groupby('Rank').agg({'Termination Probability':'min','Employee ID':'count'})

In [ ]:
df_forClustering_order_rev.groupby('Rank').agg({'Termination Probability':'max','Employee ID':'count'})

In [ ]:
df_forClustering_order_rev.to_csv('Probability of Termination.csv')

# june 20221

In [ ]:
df_june['Market Ratio Perc'] = df_june['Market Ratio']*100

In [ ]:
df_june['adj'] = df_june['Total % of adjustment']*100

In [ ]:
df_june['adj'].hist()

In [ ]:
da.list_unique_values(df_june,['Potential'])

In [ ]:
market_bin = [-100,80,95,130,1000]
market_bin_names = ['<80','80-100','100-130','>130']

adj_bin = [-100,0.2,3,1000]
adj_bin_names = ['<0.2','0.2-3','>3']

df_june['Market Ratio Cat'] = pd.cut(df_june['Market Ratio Perc'], bins = market_bin, labels = market_bin_names)
df_june['Adjustment Ratio Cat'] = pd.cut(df_june['adj'], bins = adj_bin, labels = adj_bin_names)

df_june['Potential Cat'] = df_june['Potential'].map({1:'Too Early to Rank',2:'Low Potential',3:'Medium Potential',4:'High Potential'})

featureForLogReg = ['Market Ratio Cat','Adjustment Ratio Cat','Potential Cat','Performance']
df_forClustering_june = df_june[featureForLogReg+['Employee ID']]

performance
Performance
- Developing Contributor
- Exceptional Contributor
- Not Performing
- Successful Contributor

Potential
- High
- Low
- Medium
- Too Early to Rank

In [ ]:
for col in featureForLogReg:
        df_forClustering_june = df_forClustering_june.join(pd.get_dummies(df_forClustering_june[col])) 
df_forClustering_june = df_forClustering_june.drop(featureForLogReg, axis=1)

In [ ]:
prob = clf.predict_proba(df_forClustering_june.drop(['Employee ID'], axis=1))
prob = [x for (x,y) in prob]

df_forClustering_june['Termination Probability'] = prob
df_forClustering_june

In [ ]:
df_forClustering_order_june = df_forClustering_june.sort_values(by =['Termination Probability'], ascending = False).reset_index()

In [ ]:
df_forClustering_order_june['Rank'] = pd.qcut(df_forClustering_order_june.index, 5, labels = False)

In [ ]:
df_forClustering_order_june.groupby('Rank').agg({'Termination Probability':'min','Employee ID':'count'})

In [ ]:
df_forClustering_order_june.groupby('Rank').agg({'Termination Probability':'max','Employee ID':'count'})

In [ ]:
df_probability_june = df_forClustering_order_june.groupby(by =['Rank']).agg({'Employee ID':'count'})

In [ ]:
df_forClustering_order_rev_june = df_forClustering_june.sort_values('Termination Probability', ascending = True).reset_index()
df_forClustering_order_rev_june

In [ ]:
df_forClustering_order_rev_june['Rank'] = pd.qcut(df_forClustering_order_rev_june.index, 5, labels = False)
df_forClustering_order_rev_june.groupby('Rank').agg({'Termination Probability':'min','Employee ID':'count'})

In [ ]:
df_forClustering_order_rev_june

In [ ]:
df_forClustering_order_rev_june.groupby('Rank').agg({'Termination Probability':'max','Employee ID':'count'})

In [ ]:
df_forClustering_order_rev_june.to_csv('Probability of Termination June.csv')

In [ ]:
df_forClustering_order_rev_june

# Cost Analysis

In [ ]:
new_june[['Scenario 1', 'Scenario 2',
       'Scenario 3']]

In [ ]:
Profile.columns

In [ ]:
df.merge(df_forClustering_order_rev, on = 'Employee ID')

In [ ]:
feb_merged = df.merge(df_forClustering_order_rev, on = 'Employee ID')

In [ ]:
feb_merged[['Adjustment_Diff','Current Annualized Rate','Received Adjustment?','Final EIB Adjustment',
       'Final EIB Annualized Salary']]

In [ ]:
feb_merged['Adjustment_Diff'] = np.where(feb_merged['Adjustment_Diff'].isna(), 0,feb_merged['Adjustment_Diff'])
feb_merged['% Salary Adjustment Diff'] = ((feb_merged['Final EIB Annualized Salary'] - feb_merged['Current Annualized Rate'])/feb_merged['Current Annualized Rate'])*100

#### Ask 4

In [ ]:
feb_merged[['Employee ID','Received Adjustment?','Rank']].to_excel('Feb Colleagues with Rank and adjustment.xlsx')

#### Ask 3

In [ ]:
mapper_ask4 = {'Employee ID':'# of Employees'}

ask_4 = feb_merged.groupby(['Rank','Terminated','Received Adjustment?'])\
.agg({'Employee ID':'count', '% Salary Adjustment Diff':[np.mean,np.min,np.max]}).reset_index()\
.rename(columns = mapper_ask4)

ask_4.to_excel('Feb ranking, termination and salary adj.xlsx')

In [ ]:
ask_4

In [ ]:
feb_merged.columns

In [ ]:
june_merged['% Scenario 1 Increase'] = (june_merged['Scenario 1'])/june_merged['Current Annualized Rate']*100
june_merged['% Scenario 2 Increase'] = (june_merged['Scenario 2'])/june_merged['Current Annualized Rate']*100
june_merged['% Scenario 3 Increase'] = (june_merged['Scenario 3'])/june_merged['Current Annualized Rate']*100

In [ ]:
final_merge = june_merged.merge(feb_merged, on = 'Employee ID')
final_merge['% Adjustment_Diff'] = final_merge['Adjustment_Diff']*100

In [ ]:
final_merge['% Final EIB Adjustment'] = final_merge['Final EIB Adjustment']/final_merge['Current Annualized Rate_y']*100

In [ ]:
final_merge['% Feb Increase'] = (final_merge['Adjustment_Diff'])/final_merge['Current Annualized Rate_x']*100

In [ ]:
final_merge['% Scenario 1 Increase'] = (final_merge['Scenario 1'])/final_merge['Current Annualized Rate_x']*100
final_merge['% Scenario 2 Increase'] = (final_merge['Scenario 2'])/final_merge['Current Annualized Rate_x']*100
final_merge['% Scenario 3 Increase'] = (final_merge['Scenario 3'])/final_merge['Current Annualized Rate_x']*100

In [ ]:
scenario_bin = [-100,0.1,3,8,1000]
scenario_bin_names = ['>=0','>0 and <=3','>3 and <=8','>8']

In [ ]:
mapping = {'Management Board_x':'Management Board','Rank_y':'Feb Termination Rank',
                   'Employee ID':'# of Colleagues'}

final_merge['% Final EIB Adjustment Cat'] = pd.cut(final_merge['% Final EIB Adjustment'], bins = scenario_bin, labels = scenario_bin_names)
df_agg_feb_adjust = final_merge.groupby(['Management Board_x','Rank_y','% Final EIB Adjustment Cat'])\
.agg({'Employee ID':'count'}).reset_index()\
.rename(columns = mapping)
df_agg_feb_adjust.to_excel('Termination Rank per Management board - Feb adjustment.xlsx')

In [ ]:
df_agg_feb_adjust

In [ ]:
final_merge['Scenario 1 Increase Cat'] = pd.cut(final_merge['% Scenario 1 Increase'], bins = scenario_bin, labels = scenario_bin_names)

In [ ]:
final_merge['Scenario 2 Increase Cat'] = pd.cut(final_merge['% Scenario 2 Increase'], bins = scenario_bin, labels = scenario_bin_names)

In [ ]:
final_merge['Scenario 3 Increase Cat'] = pd.cut(final_merge['% Scenario 3 Increase'], bins = scenario_bin, labels = scenario_bin_names)

In [ ]:
mapping_1 = {'Management Board_x':'Management Board','Rank_x':'June Termination Rank',
                   'Employee ID':'# of Colleagues'}

df_agg_scenaro_1 = final_merge.groupby(['Management Board_x','Rank_x','Scenario 1 Increase Cat'])\
.agg({'Employee ID':'count'}).reset_index()\
.rename(columns = mapping_1)
df_agg_scenaro_1.to_excel('Termination Rank per Management board - Scenario 1.xlsx')

In [ ]:
df_agg_scenaro_1

In [ ]:
df_agg_scenaro_2

In [ ]:
df_agg_scenaro_2 = final_merge.groupby(['Management Board_x','Rank_x','Scenario 2 Increase Cat'])\
.agg({'Employee ID':'count'}).reset_index()\
.rename(columns = mapping_1)

df_agg_scenaro_2.to_excel('Termination Rank per Management board - Scenario 2.xlsx')

df_agg_scenaro_3 = final_merge.groupby(['Management Board_x','Rank_x','Scenario 3 Increase Cat'])\
.agg({'Employee ID':'count'}).reset_index()\
.rename(columns = mapping_1)
df_agg_scenaro_3.to_excel('Termination Rank per Management board - Scenario 3.xlsx')

In [ ]:
df_agg = final_merge.groupby(['Management Board_x','Rank_x']).agg({'Employee ID':'count','% Scenario 1 Increase':'mean',
                                                       '% Scenario 2 Increase':'mean',
                                                       '% Scenario 3 Increase':'mean',


In [ ]:
df_clustered_merge = df_forClustering_order_rev_june.merge(df_forClustering_order_rev, on = 'Employee ID')

In [ ]:
management, rank, # of colleague in rank, increase, salary, 

In [ ]:
df_june.merge(df_clustered_merge, on = 'Employee ID').groupby(['Management Board','Rank']).gg({'Employee ID':'count','Current Annualized Rate':'mean',
                  'Final EIB Adjustment':'mean'})

In [ ]:
Profile = df.merge(df_forClustering_order_rev, on = 'Employee ID')
Profile

In [ ]:
Profile_agg = Profile.groupby('Management Board')\
            .agg({'Terminated':'mean',
                  'Termination Probability':'mean',
                  'Current Annualized Rate':'mean',
                  'Final EIB Adjustment':'mean'})\
            .sort_values(by = ['Termination Probability'],ascending = False)\
            .rename(columns = {'Terminated':'Profile Termination Rate',
                              'Final EIB Adjustment':'Feb EIB Adjustment, Average',
                              'Termination Probability':'Feb Termination Probability, Average',
                              'Current Annualized Rate':'Feb Annualized Rate, Average'}).reset_index()
Profile_agg

In [ ]:
Profile_june = df_june.merge(df_forClustering_order_rev_june, on = 'Employee ID')\
                .groupby(['Management Board','Rank'])\
                .agg({'Employee ID':'count','Termination Probability':'mean','Current Annualized Rate':'mean'})\
                .rename(columns = {'Employee ID':'Current # of Colleagues'})\
                .sort_values(by = ['Termination Probability'],ascending = False)\
                .reset_index()
Profile_june

In [ ]:
Profile_diff = Profile_june.merge(Profile_agg, on = ['Management Board'])\
                .rename(columns = {'Termination Probability_x':'Current Termination Probability',
                            'Current Annualized Rate_x':'Current Annualized Rate',
                            'Final Adjustment on Feb':'Final Adjustment on Feb'})

Profile_diff['Salary Increase for Profile $'] = (Profile_diff['Current Annualized Rate'] - \
                                                    Profile_diff['Feb Annualized Rate, Average'])

Profile_diff['% Salary Increase for Profile'] = (Profile_diff['Current Annualized Rate'] - \
                                                    Profile_diff['Feb Annualized Rate, Average'])/\
                                                    Profile_diff['Current Annualized Rate']*100

Profile_diff

In [ ]:
Profile_diff[Profile_diff['Pro`bfile Termination Rate'] > 0].sort_values(by =['Termination Probability','% Salary Increase for Profile'],ascending = False).to_excel('Management board termination probability.xlsx')

In [ ]:
Profile = df.merge(df_forClustering_order_rev, on = 'Employee ID')

In [ ]:
Profile_agg = Profile.groupby('Management Board')\
            .agg({'Terminated':'mean',
                  'Termination Probability':'mean',
                  'Current Annualized Rate':'mean',
                  'Final EIB Adjustment':'mean'})\
            .sort_values(by = ['Termination Probability'],ascending = False)\
            .rename(columns = {'Terminated':'Profile Termination Rate',
                              'Final EIB Adjustment':'Feb EIB Adjustment, Average',
                              'Termination Probability':'Feb Termination Probability, Average',
                              'Current Annualized Rate':'Feb Annualized Rate, Average'}).reset_index()
Profile_agg

In [ ]:
Profile_june = df_june.merge(df_forClustering_order_rev_june, on = 'Employee ID')\
                .groupby(['Management Board','Rank'])\
                .agg({'Employee ID':'count','Termination Probability':'mean','Current Annualized Rate':'mean'})\
                .rename(columns = {'Employee ID':'Current # of Colleagues'})\
                .sort_values(by = ['Termination Probability'],ascending = False)\
                .reset_index()
Profile_june

In [ ]:
Profile_diff = Profile_june.merge(Profile_agg, on = ['Management Board'])\
                .rename(columns = {'Termination Probability_x':'Current Termination Probability',
                            'Current Annualized Rate_x':'Current Annualized Rate',
                            'Final Adjustment on Feb':'Final Adjustment on Feb'})

Profile_diff['Salary Increase for Profile $'] = (Profile_diff['Current Annualized Rate'] - \
                                                    Profile_diff['Feb Annualized Rate, Average'])

Profile_diff['% Salary Increase for Profile'] = (Profile_diff['Current Annualized Rate'] - \
                                                    Profile_diff['Feb Annualized Rate, Average'])/\
                                                    Profile_diff['Current Annualized Rate']*100

Profile_diff

In [ ]:
Profile_diff[Profile_diff['Pro`bfile Termination Rate'] > 0].sort_values(by =['Termination Probability','% Salary Increase for Profile'],ascending = False).to_excel('Management board termination probability.xlsx')

In [ ]:
how many # colleague # count per adj cat

In [ ]:
how many supposed get

#### Ask 1: june add rank

In [ ]:
June_Rank = new_june.merge(df_forClustering_order_rev_june[['Rank','Employee ID']], on = 'Employee ID')
June_Rank.to_excel('June Adjument plus Rank.xlsx')

#### Ask 2 : Active vs terminated in each rank

In [ ]:
df_colleague.shape

In [ ]:
da.list_unique_values(df_colleague,['Terminated'])

In [ ]:
df_colleague['Terminated'].value_counts()

In [ ]:
filter_term_cat = (df_colleague['Termination Category'] != 'Involuntary')
df_colleague_new = df_colleague[filter_term_cat]

In [ ]:
df_colleague_new['Terminated'].value_counts()

In [ ]:
termination_filter = df_colleague_new['Termination Date'] >= '2022-02-01'
active_filter = df_colleague_new['Terminated'].isna()

In [ ]:
Terminated_colleague = df_colleague_new[termination_filter] 
Active_colleague = df_colleague_new[active_filter]

In [ ]:
print(Terminated_colleague.shape)
print(Active_colleague.shape)

In [ ]:
colleague_final = pd.concat([Terminated_colleague,Active_colleague])

In [ ]:
colleague_final.shape

In [ ]:
June_col_merged = June_Rank.merge(colleague_final[['ID - Win','Terminated','Termination Date']], left_on = 'Employee ID', right_on = 'ID - Win', how = 'left')

In [ ]:
June_col_merged['Terminated new'] = np.where(June_col_merged['Terminated'] == 'Yes', 1, 0)

In [ ]:
June_col_merged.shape

In [ ]:
da.list_unique_values(June_col_merged,['Terminated new'])

In [ ]:
June_col_merged,['Terminated']

In [ ]:
June_col_merged['Terminated new'].value_counts()

In [ ]:
mapper_3 = {'Employee ID':'# of Colleagues'}

ask_2 = June_col_merged.groupby(by = ['Rank','Terminated new'])['Employee ID'].count()\
.reset_index().rename(columns = mapper_3)


ask_2.to_excel('Terminated and active per rank.xlsx')

In [ ]:
ask_2

In [ ]:
df_forClustering.sort_values(['Termination Probability'], ascending = False)[['Termination Probability','Employee ID', 'Terminated']].to_csv('termination Probability.csv')

In [ ]:
y_true = df_forClustering_validate['Terminated']
y_predicted = clf.predict(df_forClustering_validate.drop(['Employee ID','Terminated'], axis=1))

tn, fp, fn, tp =  confusion_matrix(y_true,y_predicted).ravel()

In [ ]:
df_forClustering_validate['Terminated'].value_counts()

In [ ]:
dicCofVal = dict()
for x in range(len(df_forClustering_x.columns)):
    dicCofVal[df_forClustering_x.columns[x]] = list(clf.coef_[0])[x]

len(df_forClustering_x.columns)

df_coefValues = pd.DataFrame(columns = df_forClustering_x.columns)
df_coefValues = df_coefValues.append(dicCofVal, ignore_index=True).T
df_coefValues.columns = ['coef']

df_coefValues['odds'] = df_coefValues['coef'].apply(np.exp)
df_coefValues['prob'] = df_coefValues['odds'] /(1+df_coefValues['odds'])

featureClass = list()
for x in df_coefValues.index:
    foundMatch = False
    for feature in featureForLogReg:
        if feature in df_final.columns:
            if x in df_final[feature].unique():
                featureClass.append(feature)
                foundMatch = True

        if feature in df_forClustering.columns:
            if x in df_forClustering[feature].unique():
                featureClass.append(feature)
                foundMatch = True

    if not foundMatch:
        featureClass.append('NotFound')

df_coefValues['featureClass'] = featureClass


In [ ]:
df_coefValues.index

In [ ]:
df_coefValues.sort_values(by = ['coef'], ascending = False)[0:3].rename({'<0.2':'Less than 0.2% Salary Adjustment','<80':'Less than 80% Market Ratio'})

In [ ]:
df_coefValues.sort_values(by = ['coef'], ascending = False)[-3:].rename({'<0.2':'Less than 0.2% Salary Adjustment','<80':'Less than 80% Market Ratio'})

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = positive_corr.index
students = positive_corr['coef']
ax.bar(langs,students)
plt.show()

# Termination Proability vs features

In [ ]:
df_active = df_final.merge(listOfActiveColleagues[['Employee ID','Termination Probability']], on = 'Employee ID')

In [ ]:
plt.plot(np.sort(listOfActiveColleagues['Termination Probability'])[::-1])

In [ ]:
probability_bin = [-100,0.2,0.4,0.6,0.8,100]
probability_bin_names = ['<20','20-40','40-60','60-80','>']
df_final['adj_cat'] = pd.cut(df_final['Adjustment Difference'], bins = adj_bin, labels = adj_bin_names)

In [ ]:
df_active_potential = df_active.groupby('Potential').agg({'Employee ID':'count','HIPO + HIPER':'mean','Hot Skill':'mean','2021 Top Talent':'mean','Adjustment Difference':'mean','Termination Probability':'mean'})
display(df_active_potential)

In [ ]:
df_active_perform = df_active.groupby('Performance').agg({'Employee ID':'count','HIPO + HIPER':'mean','Hot Skill':'mean','2021 Top Talent':'mean','Adjustment Difference':'mean','Termination Probability':'mean'})
display(df_active_perform)

In [ ]:
table = pd.pivot_table(df_active, values = 'Termination Probability', index = ['Potential'], columns =['Performance'], aggfunc = np.mean)
table

# Results

In [ ]:
listOfActiveColleagues.to_excel(r"gs://ppl-analytics-cfs/Survival Ad-hoc/Termination Probability.xlsx")


# Logistic Regression with WoE

In [ ]:
df_final.columns

In [ ]:
df_withWOE = df_final.merge(df_woe_adjust[['woe']].reset_index(), on='adj_cat', how='left')
df_withWOE.rename(columns = {'woe':'woe_adjustment'}, inplace=True)

df_withWOE = df_withWOE.merge(df_woe_market[['woe']].reset_index(), on='market_cat', how='left')
df_withWOE.rename(columns = {'woe':'woe_market'}, inplace=True)


In [ ]:
df_withWOE[['Employee ID','woe_market','woe_adjustment','Terminated']]

In [ ]:
df_withWOE.replace([np.inf, -np.inf], np.nan, inplace=True)

print(df_withWOE.shape)
df_withWOE.dropna(inplace = True)
print(df_withWOE.shape)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
df_withWOE.shape

In [ ]:
df_withWOE

In [ ]:
xFeatures = columnsToConsiderForLR = ['woe_adjustment','woe_market']

# ['Market Ratio Cat','Adjustment Ratio Cat','Potential Cat','Performance']
df_train = df_withWOE.groupby('Terminated', group_keys=False).apply(lambda x: x.sample(3))


df_withWOE_forLR_train_x = df_train[xFeatures]
df_withWOE_forLR_train_y = df_train['Terminated']

clf1 = LogisticRegression(random_state=0)
clf1.fit(df_withWOE_forLR_train_x,df_withWOE_forLR_train_y)



dicCofVal = dict()

for x in range(len(df_withWOE_forLR_train_x.columns)):
    dicCofVal[df_withWOE_forLR_train_x.columns[x]] = list(clf1.coef_[0])[x]


df_coefValues = pd.DataFrame(columns =df_withWOE_forLR_train_x.columns)
df_coefValues = df_coefValues.append(dicCofVal, ignore_index=True).T
df_coefValues.columns = ['coef']

df_coefValues['odds'] = df_coefValues['coef'].apply(np.exp)
df_coefValues['prob'] = df_coefValues['odds'] /(1+df_coefValues['odds'])

df_coefValues